<a href="https://colab.research.google.com/github/voroninvisuals/google-colab-notebooks/blob/main/Deep_Keying_Matte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%pip install pytorch torchvision opencv
%pip install --upgrade pip
%pip install opencv-python
%pip install timm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.7/548.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 22.1 MB/s eta 0:00:00


In [ ]:
!pip install --quiet av pims tqdm

     |████████████████████████████████| 30.7 MB 1.1 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 


In [ ]:
import torch

model = torch.hub.load("PeterL1n/RobustVideoMatting", "mobilenetv3").cuda() # or "resnet50"
convert_video = torch.hub.load("PeterL1n/RobustVideoMatting", "converter")

In [ ]:
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from inference_utils import VideoReader, VideoWriter

reader = VideoReader('A001C031_220925_R4JU.mp4', transform=ToTensor())
writer = VideoWriter('A001C031_220925_R4JU_output.mp4', frame_rate=24)

bgr = torch.tensor([.47, 1, .6]).view(3, 1, 1).cuda()  # Green background.
rec = [None] * 4                                       # Initial recurrent states.
downsample_ratio = 0.25                                # Adjust based on your video.

with torch.no_grad():
    for src in DataLoader(reader):                     # RGB tensor normalized to 0 ~ 1.
        fgr, pha, *rec = model(src.cuda(), *rec, downsample_ratio)  # Cycle the recurrent states.
        com = fgr * pha + bgr * (1 - pha)              # Composite to green background. 
        writer.write(com)                              # Write frame.

In [ ]:
# Load the model.
model = torch.hub.load("PeterL1n/RobustVideoMatting", "resnet50") # or "resnet50"

# Converter API.
convert_video = torch.hub.load("PeterL1n/RobustVideoMatting", "converter")

In [ ]:
from inference import convert_video

convert_video(
    model,                           # The model, can be on any device (cpu or cuda).
    input_source='A001C031_220925_R4JU.v01.mp4',        # A video file or an image sequence directory.
    output_type='video',             # Choose "video" or "png_sequence"
    #output_composition='A001C031_220925_R4JU_com.mp4',    # File path if video; directory path if png sequence.
    output_alpha="/out/A001C031_220925_R4JU_v01_alpha.mp4",          # [Optional] Output the raw alpha prediction.
    #output_foreground="A001C031_220925_R4JU_fgr.mp4",     # [Optional] Output the raw foreground prediction.
    output_video_mbps=4,             # Output video mbps. Not needed for png sequence.
    downsample_ratio=0.6,           # A hyperparameter to adjust or use None for auto.
    seq_chunk=12,                    # Process n frames at once for better parallelism.
)

In [ ]:
from inference import convert_video

convert_video(
    model,                           # The loaded model, can be on any device (cpu or cuda).
    input_source='A001C031_220925_R4JU.v02.mp4',         # A video file or an image sequence directory.
    input_resize=(3424, 2202),       # [Optional] Resize the input (also the output).
    downsample_ratio=0.4,           # [Optional] If None, make downsampled max size be 512px.
    output_type='video',             # Choose "video" or "png_sequence"
    #output_composition='com.mp4',    # File path if video; direcКtory path if png sequence.
    output_alpha="./out/A001C031_220925_R4JU.v02_alpha.mp4",          # [Optional] Output the raw alpha prediction.
    #output_foreground="fgr.mp4",     # [Optional] Output the raw foreground prediction.
    output_video_mbps=8,             # Output video mbps. Not needed for png sequence.
    seq_chunk=12,                    # Process n frames at once for better parallelism.
    num_workers=1,                   # Only for image sequence input. Reader threads.
    progress=True                    # Print conversion progress.
)

  0%|          | 0/43 [00:00<?, ?it/s]